# Linear algebraic chain diagram evaluation

In [5]:
using LinearAlgebra: dot
using SpecialFunctions: gamma_inc

# Restrict ourselves to integer values of α for now
function get_d_n(α::BigInt, n_max::Int)
    @assert n_max >= 1

    c_n = BigInt(1) // α
    
    p_n = Vector{Rational{BigInt}}(undef, n_max)
    p_n1 = Vector{Rational{BigInt}}(undef, n_max)
    q_n = Vector{Rational{BigInt}}(undef, n_max)
    q_n1 = Vector{Rational{BigInt}}(undef, n_max)

    # Initial conditions
    p_n[1] = - 1 // α
    q_n[1] = 0

    # g_n(α), g_n(2α)
    g_n_α = [gamma_inc(n, α)[1] for n=1:n_max]
    g_n_2α = [gamma_inc(n, 2α)[1] for n=1:n_max]
    
    # W_n
    W_n = [1 // 2^i for i=1:n_max]

    # Computation results
    d_n = Vector{Float64}(undef, n_max)
    
    for n=1:n_max
        d_n[n] = (2n) / α * (
            α * c_n +
            dot(g_n_α[1:n], p_n[1:n]) +
            dot(g_n_2α[1:n], W_n[1:n] .* q_n[1:n])
        )

        n == n_max && break

        A_n = [(i <= j+1) ? (-1)^(i-j+1) : 0 for i=1:(n+1), j=1:n]
        B_n = [(i <= j+1) ? -1 : 0 for i=1:(n+1), j=1:n]
        U_n = [(i == 1) ? (-1)^(j-1) : 0 for i=1:(n+1), j=1:n]
        V_n = [(i == 1) ? 1 : 0 for i=1:(n+1), j=1:n]

        p_n1[1:n+1] = A_n * p_n[1:n] + V_n * q_n[1:n]
        p_n1[1] -= c_n
        q_n1[1:n+1] = B_n * q_n[1:n] + U_n * p_n[1:n]
        q_n1[1] += c_n // 2
        
        p_n[1:n+1] = (2n+1)*2n // (α^2) * p_n1[1:n+1]
        q_n[1:n+1] = (2n+1)*2n // (α^2) * q_n1[1:n+1]
        c_n = c_n * (2n+1)*2n // (2α^2)
    end

    return d_n
end

@show get_d_n(big(1), 20)

get_d_n(big(1), 20) = [0.7357588823428847, 0.4702248756489076, 0.38123919564475617, 0.33832491319786717, 0.3134077810702321, 0.2972285737620779, 0.28591192689293915, 0.2775666008227256, 0.2711648357726401, 0.2661018388852744, 0.26199921535556575, 0.2586084132389676, 0.25575958085437395, 0.2533327786347073, 0.25124094067300673, 0.24941935695122644, 0.24781894397437107, 0.24640180430464295, 0.24513821665406524, 0.24400454692139462]


20-element Vector{Float64}:
 0.7357588823428847
 0.4702248756489076
 0.38123919564475617
 0.33832491319786717
 0.3134077810702321
 0.2972285737620779
 0.28591192689293915
 0.2775666008227256
 0.2711648357726401
 0.2661018388852744
 0.26199921535556575
 0.2586084132389676
 0.25575958085437395
 0.2533327786347073
 0.25124094067300673
 0.24941935695122644
 0.24781894397437107
 0.24640180430464295
 0.24513821665406524
 0.24400454692139462

# Integral recurrence for $d_n(x)$

In [62]:
using Polynomials
using LinearAlgebra: dot

function get_d_n_from_int_x(α::Real, n_max::Int, degree::Int)
    d_n = Vector{Float64}(undef, n_max)
    
    # Coefficients of polynomial interpolation of d_1(α, x)
    p = BigFloat[i>=2 ? (2 * (-1)^i * α^(i-2) / factorial(big(i))) : 0 for i in 0:degree]
    d_n[1] = Polynomial(p)(1.0)

    # Operational matrix of the integral operator K(x-x')
    Kmat = zeros(BigFloat, degree + 1, degree + 1)
    for k in 0:degree
        for i in 1:(degree - k)
            f = prod(big(k+j) for j in 1:i)
            Kmat[k + i + 1, k + 1] = (-1)^(i-1) * α^(i-2) / f * (1.0 - 2^(i-1))
        end
    end
    
    for n in 2:n_max
        p = 2n * (2n-1) * Kmat * p
        d_n[n] = Polynomial(p)(1.0)
    end

    return d_n
end

get_d_n_from_int_x(1.0, 100, 300)

100-element Vector{Float64}:
 0.7357588823428847
 0.4702248756489076
 0.38123919564475617
 0.33832491319786717
 0.3134077810702321
 0.2972285737620779
 0.28591192689293915
 0.2775666008227256
 0.2711648357726401
 0.2661018388852744
 0.26199921535556575
 0.2586084132389676
 0.25575958085437395
 ⋮
 0.22770798387745225
 0.22765675071166455
 0.22760665150902204
 0.2275576490373374
 0.2275097076765948
 0.22746279333263206
 0.22741687335630925
 0.2273719164677619
 0.22732789268536852
 0.22728477325909474
 0.22724253060790175
 0.22720113826093363